In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [2]:
encoder_model = models.Sequential()
encoder_model.add(layers.Conv2D(128, (5, 5),strides=(2,2), activation='relu', padding = "same", input_shape=(1024, 768, 3)))
encoder_model.add(layers.Conv2D(128, (5, 5),strides=(2,2), activation='relu', padding = "same"))
encoder_model.add(layers.Conv2D(128, (5, 5),strides=(1,1), activation='relu', padding = "same"))
encoder_model.add(layers.Conv2D(192, (5, 5),strides=(2,2), activation='relu', padding = "same"))
encoder_model.add(layers.Conv2D(192, (5, 5),strides=(2,2), activation='relu', padding = "same"))

encoder_model.compile(optimizer='adagrad',loss="CosineSimilarity",metrics="[accuracy]")
print (encoder_model.summary())


decoder_model = models.Sequential()
decoder_model.add(layers.Conv2DTranspose(192, kernel_size=5, strides=2, activation='relu', padding='same', dilation_rate=(1, 1)))
decoder_model.add(layers.Conv2DTranspose(128, kernel_size=5, strides=2, activation='relu', padding='same', dilation_rate=(1, 1)))
decoder_model.add(layers.Conv2DTranspose(128, kernel_size=5, strides=1, activation='relu', padding='same', dilation_rate=(1, 1)))
decoder_model.add(layers.Conv2DTranspose(128, kernel_size=5, strides=2, activation='relu', padding='same', dilation_rate=(1, 1)))
decoder_model.add(layers.Conv2DTranspose(3,5,strides=2,padding='same',dilation_rate=(1, 1),groups=1,activation='relu',use_bias=True))
decoder_model.build(input_shape=(None, 64, 48, 192))

decoder_model.compile(optimizer='adagrad',loss="CosineSimilarity",metrics="[accuracy]")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 512, 384, 128)     9728      
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 192, 128)     409728    
                                                                 
 conv2d_2 (Conv2D)           (None, 256, 192, 128)     409728    
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 96, 192)      614592    
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 48, 192)       921792    
                                                                 
Total params: 2,365,568
Trainable params: 2,365,568
Non-trainable params: 0
_________________________________________________________________
None


In [3]:
input_img = tf.keras.layers.Input(shape=(1024, 768, 3))
encoded = encoder_model(input_img)
decoded = decoder_model(encoded)

autoencoder = tf.keras.Model(inputs=input_img, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='CosineSimilarity', metrics=['accuracy'])

In [4]:
import glob
import random
import tensorflow as tf


data_path = "C:\d_drive\Binod Sir Project\image_dataset\dataset"
train_ratio = 0.8  # Ratio of training data to total data
val_ratio = 0.2  # Ratio of validation data to total data

file_paths = glob.glob(data_path + '/*.jpg')
batch_size = 16
random.shuffle(file_paths)

def read_image(file_path):
    image = tf.io.read_file(file_path)
    return image

def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, (1024, 768))
    return image, image  # duplicate the image to use as both input and target

# split_index_train = int(len(file_paths) * train_ratio)
# split_index_val = int(len(file_paths) * (train_ratio + val_ratio))

# train_file_paths = file_paths[:split_index_train]
# val_file_paths = file_paths[split_index_train:split_index_val]

img_dataset = tf.data.Dataset.list_files(file_paths)
# train_dataset = tf.data.Dataset.from_tensor_slices(train_file_paths)
# val_dataset = tf.data.Dataset.from_tensor_slices(val_file_paths)
img_dataset = img_dataset.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
img_dataset = img_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
img_dataset = img_dataset.batch(batch_size)
img_dataset = img_dataset.prefetch(tf.data.AUTOTUNE)
# train_dataset = train_dataset.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
# train_dataset = train_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
# train_dataset = train_dataset.batch(batch_size)
# train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
# train_dataset = train_dataset.repeat()

# Load and preprocess the validation dataset
# val_dataset = val_dataset.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
# val_dataset = val_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
# val_dataset = val_dataset.batch(batch_size)
# val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

In [6]:
autoencoder.fit(img_dataset, epochs=5)

Epoch 1/5
 11/659 [..............................] - ETA: 32:56:03 - loss: -0.8766 - accuracy: 0.6079

KeyboardInterrupt: 

In [ ]:
autoencoder.save("C:\d_drive\Binod Sir Project\model_epoch=5_loss=cs")

In [2]:
import tensorflow as tf
print ("GPU Avilable:", tf.config.list_physical_devices('GPU'))

GPU Avilable: []


In [4]:
import torch

print("cuDNN version:", torch.backends.cudnn.version())


cuDNN version: 8500


In [6]:
import torch

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Number of available GPUs: {device_count}")
    for i in range(device_count):
        device_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {device_name}")
else:
    print("No GPUs available.")


Number of available GPUs: 1
GPU 0: NVIDIA GeForce RTX 3050 Laptop GPU
